# Language Modeling

## Part 1 - Individual

We are interested in building a language model over a language with three words: A, B, C. Our training corpus is

In [1]:
train_corpus = "AAABACBABBBCCACBCC"

### Problem 1

**First train a unigram language model using maximum likelihood estimation. What are the probabilities? (Just leave in the form of a fraction)? Reminder: We don't need start or end tokens for training a unigram model, since the context of each word doesn't matter. So, we will not add any special tokens to our corpus for this part of the problem.**

In [2]:
import numpy as np
import pandas as pd
from collections import Counter

unigram_counts = Counter(train_corpus)

print(unigram_counts)
print("Total corpus length is %d" % len(train_corpus))

Counter({'A': 6, 'C': 6, 'B': 6})
Total corpus length is 18


**P(A) = $\frac{6}{18} = \frac{1}{3}$**

**P(B) = $\frac{1}{3}$**

**P(C) = $\frac{1}{3}$**

### Problem 2
**Next train a bigram language model using maximum likelihood estimation. For this problem, we will add an end token, $, at the end of the string, so that we can model the probability of the sentence ending after a particular word. If you chose to add a start token as well, that's fine too, but these solutions assume no start token. Fill in the probabilities below. Leave your answers in the form of a fraction.**

In [3]:
# add end token to corpus
extended_train_corpus = train_corpus + '$'
print(extended_train_corpus)

# set up bigram table with zeros for every possible bigram from our corpus
bigram_counts = {(token1, token2):0 for token1 in train_corpus for token2 in extended_train_corpus}

# count bigrams that actually occur within our corpus
corpus_bigram_counts = Counter(zip(train_corpus, extended_train_corpus[1:]))

# add these bigram counts to our bigram table
for bigram in corpus_bigram_counts.keys():
    bigram_counts[bigram] += corpus_bigram_counts[bigram]

print(bigram_counts)

AAABACBABBBCCACBCC$
{('B', 'C'): 2, ('A', 'A'): 2, ('B', 'B'): 2, ('B', 'A'): 2, ('C', 'A'): 1, ('A', '$'): 0, ('C', 'C'): 2, ('C', 'B'): 2, ('A', 'B'): 2, ('A', 'C'): 2, ('C', '$'): 1, ('B', '$'): 0}


Each bigram probability should be the bigram count divided by the unigram count of the first token. In our corpus, every unigram count is 6.

**P(A|A) = $\frac{2}{6} = \frac{1}{3}$**

**P(A|B) = $\frac{2}{6} = \frac{1}{3}$**

**P(A|C) = $\frac{1}{6}$**

**P(B|A) = $\frac{2}{6} = \frac{1}{3}$**

**P(B|B) = $\frac{2}{6} = \frac{1}{3}$**

**P(B|C) = $\frac{2}{6} = \frac{1}{3}$**

**P(C|A) = $\frac{2}{6} = \frac{1}{3}$**

**P(C|B) = $\frac{2}{6} = \frac{1}{3}$**

**P(C|C) = $\frac{2}{6} = \frac{1}{3}$**

### Problem 3

**Now evaluate your language models on the corpus "ABACABB". What is the perplexity of the unigram language model evaluated on this corpus? Since we didn't add any special start/end tokens when we were training our unigram language model, we won't add any when we evaluate the perplexity of the unigram language model, either, so that we're consistent.**

In [4]:
"""
Turn unigram counts table into unigram probability table by dividing
each unigram count by the total number of words in the training corpus
"""
unigram_probs = {unigram: unigram_counts[unigram] / float(len(train_corpus)) \
                 for unigram in unigram_counts.keys()}

test_sent = "ABACABB"
N = len(test_sent)

"""
Approximate the joint probability P(w1w2...wn) on the test sentence
using the product of unigram probabilities P(w1)*P(w2)*...*P(wn)
"""
joint_prob_unigram_estimate = 1
for unigram in test_sent:
    joint_prob_unigram_estimate *= unigram_probs[unigram]
    
# perplexity = 1 / P(w1w2...wn)^(1/n)
unigram_perplexity = 1.0 / (joint_prob_unigram_estimate)**(1.0/N)
print(unigram_perplexity)

3.0


**What is the perplexity of the bigram language model evaluated on this corpus? Since we added an end token when we were training our bigram model, we'll add an end token to this corpus again before we evaluate perplexity.**

In [5]:
"""Turn bigram counts table into bigram probabilities by dividing by unigram counts"""
bigram_probs = dict(bigram_counts)  # copy
for bigram in corpus_bigram_counts.keys():  # don't have unigram counts for end token
    first_token = bigram[0]
    bigram_probs[bigram] /= float(unigram_counts[first_token])

extended_test_sent = test_sent + '$'
N = len(extended_test_sent)
    
"""
Approximate the joint probability P(w1w2...wn) on the test sentence
using the product of bigram probabilities P(w)*P(w2)*...*P(wn)
"""
joint_prob_bigram_estimate = 1
for bigram in zip(test_sent, extended_test_sent[1:]):
    joint_prob_bigram_estimate *= bigram_probs[bigram]
    
print("Joint probability bigram estimate: %f" % joint_prob_bigram_estimate)
    
# perplexity = 1 / P(w1w2...wn)^(1/n)
bigram_perplexity = 1.0 / (joint_prob_bigram_estimate)**(1.0/N)
print("Bigram language model perplexity: %f" % bigram_perplexity)

Joint probability bigram estimate: 0.000000


ZeroDivisionError: float division by zero

Ok, this Zero Division Error actually makes sense. The joint probability estimate should be 0, because the last bigram of our test sentence is 'B$'. This bigram actually never occurred in our training corpus, and so it has a count and probability of 0. Thus the perplexity is undefined with this naive model.

### Problem 4

**Now repeat everything above for add-1 (Laplace) smoothing.**

Ok, Laplace smoothing should fix our zero division error.

In [6]:
# Define relevant N and V values from the training corpus
n_train_corpus = len(train_corpus)  # N, number of tokens in training corpus
v_train_corpus = len(set(train_corpus))  # V, number of unique tokens in training corpus

# Add 1 to every unigram count
laplace_unigram_counts = {unigram:c+1 for unigram,c in unigram_counts.iteritems()}

"""
Create unigram probabilities as before, using (c + 1) / (N + V)
"""
laplace_unigram_probs = {unigram: c / float(n_train_corpus + v_train_corpus) \
                         for unigram,c in laplace_unigram_counts.iteritems()}

test_sent = "ABACABB"
N = len(test_sent)

"""
Approximate the joint probability P(w1w2...wn) on the test sentence
using the product of unigram probabilities P(w1)*P(w2)*...*P(wn)
"""
joint_prob_laplace_unigram_estimate = 1
for unigram in test_sent:
    joint_prob_laplace_unigram_estimate *= laplace_unigram_probs[unigram]
    
# perplexity = 1 / P(w1w2...wn)^(1/n)
laplace_unigram_perplexity = 1.0 / (joint_prob_laplace_unigram_estimate)**(1.0/N)  # different N (length of test sentence)
print("Laplace unigram perplexity: %f" % laplace_unigram_perplexity)

Laplace unigram perplexity: 3.000000


In [7]:
"""
Create bigram probabilities as before, using (c_bi + 1) / (c_uni + V)
"""
laplace_bigram_probs = {bigram:c+1 for bigram,c in bigram_counts.iteritems()}  # copy counts, and add-1

for bigram in corpus_bigram_counts.keys():  # don't have unigram counts for end token
    first_token = bigram[0]
    laplace_bigram_probs[bigram] /= float(unigram_counts[first_token])

extended_test_sent = test_sent + '$'
N = len(extended_test_sent)
    
"""
Approximate the joint probability P(w1w2...wn) on the test sentence
using the product of bigram probabilities P(w)*P(w2)*...*P(wn)
"""
joint_prob_laplace_bigram_estimate = 1
for bigram in zip(test_sent, extended_test_sent[1:]):
    joint_prob_laplace_bigram_estimate *= laplace_bigram_probs[bigram]
    
print("Joint probability laplace bigram estimate: %f" % joint_prob_laplace_bigram_estimate)
    
# perplexity = 1 / P(w1w2...wn)^(1/n)
laplace_bigram_perplexity = 1.0 / (joint_prob_laplace_bigram_estimate)**(1.0/N)
print("Laplace bigram language model perplexity: %f" % laplace_bigram_perplexity)

Joint probability laplace bigram estimate: 0.010417
Laplace bigram language model perplexity: 1.769228


We can see that laplace-smoothed bigram language model is less perplexed than the unigram model, indicating it is the better choice for this test sentence.

## Part 2 - Group Work with Person X, Person Y

### Problem 1

**What is the difference between using an UNK token (for unknown words) and smoothing?**

**Give an example where you would use one versus the other.**

### Problem 2

**Suppose you build an interpolated trigram language model, with three weights $\lambda_1$ for unigrams, $\lambda_2$ for bigrams, and $\lambda_3$ for trigrams. Normally we set these lambdas on a held-out set. Suppose instead we set them on the training data. This will cause the lambdas to take on very unusual values. What will these lambdas look like? Why?**

### Problem 3

**Show that if we estimate two bigram language models using unsmoothed relative frequencies (MLE), one from a text corpus and the second from the same corpus in reverse order, the models will assign the same probability to new sentences (when applied in forward and backward order respectively). Hint: First write out the entire equation for sentence probabilities in terms of counts.**